# Automated experiments

This file contains an extended version of the code shown in continuous_variable_training.ipynb. Several options are added, such as convolutional layers, Gaussian kernels, and the ability to use a variational autoencoder. Methods were written so that a queue of experiments can be made, while the amount of outputs is heavily reduced.

After running the experiments, the data was copied into a text editor, and using regular expressions, lines that do not contain curly brackets (such lines only contain information which is useful to see training progress) were removed. This data can be pasted into a program such as Google Sheets or Microsoft Excel, and split into multiple columns (like in a .csv file) using the semicolon as delimiter. The result of this can be seen in the .xlsx file included in this folder.

For detailed explanations on the code, please look at continuous_variable_training.ipynb

PS. these experiments will take several days to run on modern desktop computers. You might want to comment some of the lines in the second code cell to reduce the amount of measurements.

In [ ]:

get_ipython().run_line_magic('matplotlib', 'inline')
# from pylab import *
import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib
import scipy.stats
import scipy.spatial
import tensorflow as tf
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
# from PIL import Image
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import sklearn.model_selection
import tensorflow.keras as keras
import os
import neural_structured_learning as nsl
import math
import gc
import gkernel
# import gkernel
# import functools

print('Imports done')

In [ ]:

def gen_experiment(input_dict={},variable=None,vars=None):
    if variable is None:
        vars = [0]
    experiment=[]
    for x in vars:
        new_experiment_config=input_dict.copy()
        if variable is not None:
            new_experiment_config[variable]=x
        new_experiment_config['string']=str(new_experiment_config)
        experiment.append(new_experiment_config)
    return experiment

experiments = []






# JSD experiments
experiments.append(gen_experiment({'loss_function':'JSD'}))
experiments.append(gen_experiment({'loss_function':'JSD'},'sampling_density',[4,15,25,50,100,150,300]))
experiments.append(gen_experiment({'loss_function':'JSD'},'hidden_layers',[3,5,7,9,27]))
experiments.append(gen_experiment({'loss_function':'JSD'},'encoding_dim',[2,3,6]))
experiments.append(gen_experiment({'loss_function':'JSD'},'activation_types',[ ['relu'],['relu','relu','relu','relu','relu'], [keras.backend.sin,keras.backend.cos,keras.activations.linear],[keras.backend.sin,keras.backend.cos,keras.activations.linear, 'relu', 'sigmoid']]))
experiments.append(gen_experiment({'loss_function':'JSD'},'activity_regularizer',[keras.regularizers.l2(0.01),keras.regularizers.l2(10e-5),keras.regularizers.l1(0.01),keras.regularizers.l1(10e-5)]))
experiments.append(gen_experiment({'loss_function':'JSD'},'sigma',[0.01*4/4,0.01/4,0.01/math.sqrt(4),0.01*4/100,0.01/100,0.01/math.sqrt(100)]))
experiments.append(gen_experiment({'loss_function':'JSD'},'input_layer_type',['gaussian_noise','gaussian_dropout','sqrt_softmax']))
experiments.append(gen_experiment({'loss_function':'JSD'},'unlabeled_data_percentage',[1-(5/100),1-(1/100),1-(0.5/100),1-(0.25/100),1-(0.125/100)]))
experiments.append(gen_experiment({'loss_function':'JSD'},'BN_size',[2,3,4,5]))
# JSD experiments NB100
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100}))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100},'hidden_layers',[3,5,7,9,27]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100},'encoding_dim',[2,3,6]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100},'activation_types',[ ['relu'],['relu','relu','relu','relu','relu'], [keras.backend.sin,keras.backend.cos,keras.activations.linear],[keras.backend.sin,keras.backend.cos,keras.activations.linear, 'relu', 'sigmoid'] ]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100},'activity_regularizer',[keras.regularizers.l2(0.01),keras.regularizers.l2(10e-5),keras.regularizers.l1(0.01),keras.regularizers.l1(10e-5)]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100},'sigma',[0.01*4/4,0.01/4,0.01/math.sqrt(4),0.01*4/100,0.01/100,0.01/math.sqrt(100)]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100},'input_layer_type',['gaussian_noise','gaussian_dropout','sqrt_softmax']))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100},'unlabeled_data_percentage',[1-(5/100),1-(1/100),1-(0.5/100),1-(0.25/100),1-(0.125/100)]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100},'BN_size',[2,3,4,5]))
# JSD Gaussian kernel experiments
experiments.append(gen_experiment({'loss_function':'JSD','input_layer_type':'gaussian_kernel'},'kernel_landmarks',[25,50,100,200,500,1000]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100,'input_layer_type':'gaussian_kernel'},'kernel_landmarks',[25,50,100,200,500,1000]))
# CCE Gaussian kernel experiments
experiments.append(gen_experiment({'input_layer_type':'gaussian_kernel'},'kernel_landmarks',[25,50,100,200,500,1000]))
experiments.append(gen_experiment({'sampling_density':100,'input_layer_type':'gaussian_kernel'},'kernel_landmarks',[25,50,100,200,500,1000]))


# CCE experiments
experiments.append(gen_experiment({}))
experiments.append(gen_experiment({},'sampling_density',[4,15,25,50,100,150,300]))
experiments.append(gen_experiment({},'hidden_layers',[3,5,7,9,27]))
experiments.append(gen_experiment({},'encoding_dim',[2,3,6]))
experiments.append(gen_experiment({},'activation_types',[ ['relu'],['relu','relu','relu','relu','relu'], [keras.backend.sin,keras.backend.cos,keras.activations.linear],[keras.backend.sin,keras.backend.cos,keras.activations.linear, 'relu', 'sigmoid']]))
experiments.append(gen_experiment({},'activity_regularizer',[keras.regularizers.l2(0.01),keras.regularizers.l2(10e-5),keras.regularizers.l1(0.01),keras.regularizers.l1(10e-5)]))
experiments.append(gen_experiment({},'sigma',[0.01*4/4,0.01/4,0.01/math.sqrt(4),0.01*4/100,0.01/100,0.01/math.sqrt(100)]))
experiments.append(gen_experiment({},'input_layer_type',['gaussian_noise','gaussian_dropout','sqrt_softmax']))
experiments.append(gen_experiment({},'unlabeled_data_percentage',[1-(5/100),1-(1/100),1-(0.5/100),1-(0.25/100),1-(0.125/100)]))
experiments.append(gen_experiment({},'BN_size',[2,3,4,5]))
# CCE experiments NB100
experiments.append(gen_experiment({'sampling_density':100}))
experiments.append(gen_experiment({'sampling_density':100},'hidden_layers',[3,5,7,9,27]))
experiments.append(gen_experiment({'sampling_density':100},'encoding_dim',[2,3,6]))
experiments.append(gen_experiment({'sampling_density':100},'activation_types',[ ['relu'],['relu','relu','relu','relu','relu'], [keras.backend.sin,keras.backend.cos,keras.activations.linear],[keras.backend.sin,keras.backend.cos,keras.activations.linear, 'relu', 'sigmoid'] ]))
experiments.append(gen_experiment({'sampling_density':100},'activity_regularizer',[keras.regularizers.l2(0.01),keras.regularizers.l2(10e-5),keras.regularizers.l1(0.01),keras.regularizers.l1(10e-5)]))
experiments.append(gen_experiment({'sampling_density':100},'sigma',[0.01*4/4,0.01/4,0.01/math.sqrt(4),0.01*4/100,0.01/100,0.01/math.sqrt(100)]))
experiments.append(gen_experiment({'sampling_density':100},'input_layer_type',['gaussian_noise','gaussian_dropout','sqrt_softmax']))
experiments.append(gen_experiment({'sampling_density':100},'unlabeled_data_percentage',[1-(5/100),1-(1/100),1-(0.5/100),1-(0.25/100),1-(0.125/100)]))
experiments.append(gen_experiment({'sampling_density':100},'BN_size',[2,3,4,5]))

# CCEu experiments
experiments.append(gen_experiment({'training_method':'unsupervised'},'hidden_layers',[3,5,7,9,27]))
experiments.append(gen_experiment({'training_method':'unsupervised'},'encoding_dim',[2,3,6]))
experiments.append(gen_experiment({'training_method':'unsupervised'},'sigma',[0.01*4/4,0.01/4,0.01/math.sqrt(4),0.01*4/100,0.01/100,0.01/math.sqrt(100)]))
experiments.append(gen_experiment({'training_method':'unsupervised'},'input_layer_type',['gaussian_noise','gaussian_dropout','sqrt_softmax']))
experiments.append(gen_experiment({'training_method':'unsupervised'},'BN_size',[2,3,4,5]))

# vergeten CCE
experiments.append(gen_experiment({'sampling_density':100},'unlabeled_data_percentage',[1-(5/100),1-(1/100),1-(0.5/100),1-(0.25/100),1-(0.125/100)]))
experiments.append(gen_experiment(sampling_density':100},'sigma',[0.01*4/4,0.01/4,0.01/math.sqrt(4),0.01*4/100,0.01/100,0.01/math.sqrt(100)]))
experiments.append(gen_experiment({'sampling_density':100},'activation_types',[ ['relu','relu','relu','relu','relu'] ]))
experiments.append(gen_experiment({'training_method':'unsupervised','input_layer_type':'gaussian_kernel'},'kernel_landmarks',[25,50,100,200,500,1000]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100},'hidden_layers',[3,5,7,9,27]))



# CNN experiments
experiments.append(gen_experiment({'loss_function':'JSD','CNN':True},'CNN_kernel_size',[1,3,5,7,9]))
experiments.append(gen_experiment({'loss_function':'JSD','CNN':True},'CNN_filters',[16,32,64,128]))
experiments.append(gen_experiment({'loss_function':'JSD','CNN':True,'sampling_density':100},'CNN_kernel_size',[1,3,5,7,9]))
experiments.append(gen_experiment({'loss_function':'JSD','CNN':True,'sampling_density':100},'CNN_filters',[16,32,64,128]))


experiments.append(gen_experiment({'loss_function':'JSD','CNN':True,'sampling_density':100,'CNN_kernel_size':100}))
experiments.append(gen_experiment({'loss_function':'JSD','CNN':True,'sampling_density':4,'CNN_kernel_size':4}))








# 5x relu
experiments.append(gen_experiment({'loss_function':'JSD'},'activation_types',[ ['relu','relu','relu','relu','relu'] ]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100},'activation_types',[ ['relu','relu','relu','relu','relu'] ]))

experiments.append(gen_experiment({'training_method':'unsupervised','CNN':True},'CNN_kernel_size',[1,3,5,7,9,4]))
experiments.append(gen_experiment({'training_method':'unsupervised','CNN':True},'CNN_filters',[16,32,64,128]))
experiments.append(gen_experiment({'CNN':True},'CNN_kernel_size',[1,3,5,7,9,4]))
experiments.append(gen_experiment({'CNN':True},'CNN_filters',[16,32,64,128]))
experiments.append(gen_experiment({'CNN':True,'sampling_density':100},'CNN_kernel_size',[1,3,5,7,9,100]))
experiments.append(gen_experiment({'CNN':True,'sampling_density':100},'CNN_filters',[16,32,64,128]))



experiments.append(gen_experiment({'training_method':'unsupervised'},'BN_size',[10,20,30]))
experiments.append(gen_experiment({},'BN_size',[2,3,4,5,10,20,30]))
experiments.append(gen_experiment({},'unlabeled_data_percentage',[1-(5/100),1-(1/100),1-(0.5/100),1-(0.25/100),1-(0.125/100)]))
experiments.append(gen_experiment({'loss_function':'JSD'},'BN_size',[10,20,30]))
experiments.append(gen_experiment({'training_method':'unsupervised'},'sampling_density',[4,15,25,50,100,150,300]))
experiments.append(gen_experiment({},'training_method',["supervised","supervised_2_percent","semi","semi_sup_first","semi_mixed","unsupervised"]))

# training method
experiments.append(gen_experiment({},'training_method',["semi_sup_first","semi_mixed","unsupervised"]))
experiments.append(gen_experiment({'loss_function':'JSD'},'training_method',["supervised","supervised_2_percent","semi","semi_sup_first","semi_mixed","unsupervised"]))
experiments.append(gen_experiment({'loss_function':'MSE'},'training_method',["supervised","supervised_2_percent","semi","semi_sup_first","semi_mixed","unsupervised"]))

experiments.append(gen_experiment({'sampling_density':100},'training_method',["supervised","supervised_2_percent","semi","semi_sup_first","semi_mixed","unsupervised"]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100},'training_method',["supervised","supervised_2_percent","semi","semi_sup_first","semi_mixed","unsupervised"]))
experiments.append(gen_experiment({'loss_function':'MSE','sampling_density':100},'training_method',["supervised","supervised_2_percent","semi","semi_sup_first","semi_mixed","unsupervised"]))

# noise
experiments.append(gen_experiment({'training_method':'unsupervised'},'sigma',[0.05,0.1,0.2]))
experiments.append(gen_experiment({},'sigma',[0.05,0.1,0.2]))
experiments.append(gen_experiment({'sampling_density':100},'sigma',[0.05,0.1,0.2]))
experiments.append(gen_experiment({'loss_function':'JSD'},'sigma',[0.05,0.1,0.2]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100},'sigma',[0.05,0.1,0.2]))

#  VAE
experiments.append(gen_experiment({'training_method':'unsupervised'},'VAE',[True]))
experiments.append(gen_experiment({'sampling_density':100},'VAE',[True]))
experiments.append(gen_experiment({'loss_function':'JSD'},'VAE',[True]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100},'VAE',[True]))


# JSD5
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100,'hidden_layers':5},'input_layer_type',['gaussian_noise','gaussian_dropout','sqrt_softmax','gaussian_kernel']))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100,'hidden_layers':5,'CNN':True}))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100,'hidden_layers':5,'VAE':True}))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100,'hidden_layers':5},'activity_regularizer',[keras.regularizers.l2(0.01),keras.regularizers.l2(10e-5),keras.regularizers.l1(0.01),keras.regularizers.l1(10e-5)]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100,'hidden_layers':5},'activation_types',[ ['relu'],['relu','relu','relu','relu','relu'], [keras.backend.sin,keras.backend.cos,keras.activations.linear],[keras.backend.sin,keras.backend.cos,keras.activations.linear, 'relu', 'sigmoid']]))

# further sampling density
experiments.append(gen_experiment({'training_method':'unsupervised'},'sampling_density',[150,300]))
experiments.append(gen_experiment({'loss_function':'JSD'},'sampling_density',[150,300]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100,'hidden_layers':5,'epochs':70},'sampling_density',[300]))


# further jsd5
experiments.append(gen_experiment({'sampling_density':100},'unlabeled_data_percentage',[1-(1/100)]))

experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100,'hidden_layers':5},'encoding_dim',[2,3,6]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100,'hidden_layers':5},'sigma',[0.01*4/4,0.01/4,0.01/math.sqrt(4),0.01*4/100,0.01/100,0.01/math.sqrt(100),0.05,0.1,0.2]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100,'hidden_layers':5},'BN_size',[2,3,4,5]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100,'hidden_layers':5},'unlabeled_data_percentage',[1-(5/100),1-(1/100),1-(0.5/100),1-(0.25/100),1-(0.125/100)]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100,'hidden_layers':5,'input_layer_type':'gaussian_kernel'},'kernel_landmarks',[25,50,100,200,500,1000]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100,'hidden_layers':5,'CNN':True},'CNN_kernel_size',[1,3,5,7,9,100]))

experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100,'hidden_layers':5,'input_layer_type':'gaussian_kernel','kernel_landmarks':25}))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100,'hidden_layers':5,'input_layer_type':'gaussian_kernel','kernel_landmarks':10}))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100,'hidden_layers':5,'input_layer_type':'gaussian_kernel','kernel_landmarks':3}))

experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100,'hidden_layers':5,'CNN':True},'CNN_filters',[16,32,64,128]))


# TODO further training method

experiments.append(gen_experiment({},'training_method',["supervised","supervised_2_percent","semi_sup_first"]))
experiments.append(gen_experiment({'loss_function':'JSD'},'training_method',["supervised","supervised_2_percent","semi_sup_first"]))
experiments.append(gen_experiment({'loss_function':'MSE'},'training_method',["supervised","supervised_2_percent","semi_sup_first"]))
experiments.append(gen_experiment({'sampling_density':100},'training_method',["supervised","supervised_2_percent","semi_sup_first"]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100},'training_method',["supervised","supervised_2_percent","semi_sup_first"]))
experiments.append(gen_experiment({'loss_function':'MSE','sampling_density':100},'training_method',["supervised","supervised_2_percent","semi_sup_first"]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100,'hidden_layers':5},'training_method',["supervised","supervised_2_percent","semi","semi_sup_first","semi_mixed","unsupervised"]))



# more Gaussian kernel experiments
experiments.append(gen_experiment({'loss_function':'JSD','input_layer_type':'gaussian_kernel'},'kernel_landmarks',[3,10]))
experiments.append(gen_experiment({'loss_function':'JSD','sampling_density':100,'input_layer_type':'gaussian_kernel'},'kernel_landmarks',[3,10]))
experiments.append(gen_experiment({'input_layer_type':'gaussian_kernel'},'kernel_landmarks',[3,10]))
experiments.append(gen_experiment({'sampling_density':100,'input_layer_type':'gaussian_kernel'},'kernel_landmarks',[3,10]))
experiments.append(gen_experiment({'training_method':'unsupervised','input_layer_type':'gaussian_kernel'},'kernel_landmarks',[3,10]))





for ex in experiments:
    for config in ex:
        print(config['string'])

In [ ]:
# # Default Variables for experiments.You probably want to use the cell above to change experiment vars.
BN_size_default =3 # amount of BN variables, minimum 3

mu_default, sigma_default = 0, 0.01 # Distribution of the noise
sampling_density_default =4 # How many bins the quasi-continuous variables use for distributing their probabilities. Higher_default = better approximation of continuous distributions

activation_types_default = [keras.backend.sin,keras.backend.cos,keras.activations.linear, 'relu', 'swish'] # Activation layer types
hidden_layers_default =9 # amount of hidden layers
encoding_dim_default = BN_size_default # The dimensionality of the middle layer
loss_function_default = keras.losses.categorical_crossentropy
training_method_default ='semi'
activity_regularizer_default=None
input_layer_type_default='dense'
unlabeled_data_percentage_default=0.98

epochs_default=100
VAE_default = False
CNN_default = False
kernel_landmarks_default = 100

CNN_layers_default = 1
CNN_filters_default = 64
CNN_kernel_size_default = 3

In [ ]:
def generate_samplespace(function,x_min,x_max,measurements):
    non_normalized = [function(x) for x in np.linspace(x_min,x_max,measurements)]
    return (non_normalized / sum(non_normalized))

def normalize_df(df):
    return df.div(df.sum(axis=1), axis=0)
#     return df.subtract(df.min(axis=1), axis=0).div(df.sum(axis=1), axis=0)

def custom_loss2(y_true,y_pred,sizes_sorted):

    i=0
    total_loss = 0
    for size in sizes_sorted:
#         total_loss += keras.losses.kullback_leibler_divergence(y_true[:,i:i+size],y_pred[:,i:i+size])
        total_loss += nsl.lib.jensen_shannon_divergence(y_true[:,i:i+size],y_pred[:,i:i+size],axis=1)
        i+=size
    return total_loss

def prob_distance(x,y):
    xt=x.T # this is bad practice, but I cannot debug the statements below without doing this
    yt=y.T
    res= scipy.spatial.distance.jensenshannon(xt,yt,base=2.0)
#     print(res)
    return res

def make_bn(BN_size,sampling_density):
    bn=gum.BayesNet("Quasi-Continuous")
    a=bn.add(gum.LabelizedVariable("A","A binary variable",2))
    bn.cpt(a)[:]=[0.4, 0.6]

    if BN_size>1:
        b=bn.add(gum.RangeVariable("B","A range variable",0,sampling_density-1))
        bn.addArc(a,b)
        first = generate_samplespace(scipy.stats.norm().pdf,-10,3,sampling_density)
        second = generate_samplespace(scipy.stats.norm().pdf,-2,6,sampling_density)
        bn.cpt(b)[{'A':0}]=first
        bn.cpt(b)[{'A':1}]=second


    if BN_size>2:
        c=bn.add(gum.RangeVariable("C","Another quasi continuous variable",0,sampling_density-1))
        bn.addArc(b,c)
        l=[]
        for i in range(sampling_density):
            # the size and the parameter of gamma depends on the parent value
            k=(i*30.0)/sampling_density
            l.append(generate_samplespace(scipy.stats.gamma(k+1).pdf,4,5+k,sampling_density))
        bn.cpt(c)[:]=l


        for d in range(BN_size-3):
            # new variable
            d=bn.add(gum.RangeVariable("D"+str(d),"Another quasi continuous variable",0,sampling_density-1))
            l=[]
            bn.addArc(c,d)
            for i in range(sampling_density):
                # the size and the parameter of gamma depends on the parent value
                k=(i*30.0)/sampling_density
                l.append(generate_samplespace(scipy.stats.gamma(k+1).pdf,4,5+k,sampling_density))
            bn.cpt(d)[:]=l

    bn.cpt(a)[:]=[0.1,0.9]
    return bn

def make_df(use_previous_df,bn,mu,sigma):
    if use_previous_df:
        original_database=pd.read_csv("good_db.csv")
    else:
        gum.generateCSV(bn,"database.csv",10000)
        original_database=pd.read_csv("database.csv")
    original_database = original_database.reindex(sorted(original_database.columns), axis=1)


    size_dict = {}
    for column_name in original_database.columns:
        size_dict[column_name] = bn.variable(column_name).domainSize()

    shape = [original_database.shape[0],sum(size_dict.values())]

    df_cols_sorted = sorted(list(original_database.columns))
    sizes_sorted = [size_dict[x] for x in df_cols_sorted]
    sizes_sorted_with_leading_zero = [0] + sizes_sorted

    data=np.ones(original_database.shape[0]*original_database.shape[1])
    row = list(range(original_database.shape[0]))*original_database.shape[1]
    col = []
    for i in range(original_database.values.T.shape[0]):
        for item in original_database.values.T[i]:
            col.append(item+sum(sizes_sorted_with_leading_zero[0:i+1]))
    # print(col[20000])

    
    input3 = scipy.sparse.coo_matrix((data, (row, col)), shape=tuple(shape)).todense()

    first_id2 = df_cols_sorted[:]
    second_id2=[list(range(x)) for x in sizes_sorted]

    arrays3=[np.repeat(first_id2,sizes_sorted) , [item for sublist in second_id2 for item in sublist]]
    tuples2 = list(zip(*arrays3))
    index2 = pd.MultiIndex.from_tuples(tuples2, names=['Variable', 'Value'])

    hard_evidence = pd.DataFrame(input3,columns=index2)

    noise = np.random.normal(mu, sigma, hard_evidence.shape) 
    df = hard_evidence + noise
    df = df.clip(lower=0)
    
    for col in df_cols_sorted:
        df[col] = normalize_df(df[col])
    
    return df, hard_evidence, sizes_sorted

# def vae_loss(z_mean, z_log_var,loss_func):
#     # adapted from https://stackoverflow.com/a/61945712
#     def loss(y_true, y_pred):
#         # mse loss
#         reconstruction_loss = loss_func(y_true, y_pred)
#         # kl loss
#         kl_loss = 1 + z_log_var - keras.backend.square(z_mean) - keras.backend.exp(z_log_var)
#         kl_loss = keras.backend.sum(kl_loss, axis=-1)
#         kl_loss *= -0.5
#         weight = 0.
#         return reconstruction_loss + (weight * kl_loss)
#     return loss

# def sampling(args):
#     # adapted from https://blog.keras.io/building-autoencoders-in-keras.html
#     z_mean, z_log_var = args
#     #     batch = keras.backend.shape(z_mean)[0]
#     # dim = keras.backend.int_shape(z_mean)[1]
#     # epsilon = keras.backend.random_normal(shape=(batch, dim))
#     batch = z_mean.shape[0]
#     dim = z_mean.shape[1]

#     # by default, random_normal has mean = 0 and std = 1.0
#     epsilon = keras.backend.random_normal(shape=(dim,))
#     thing = z_mean + keras.backend.exp(0.5 * z_log_var) * epsilon
#     return thing

class KLDivergenceLayer(keras.layers.Layer):

    """ Identity transform layer that adds KL divergence
    to the final model loss.
    """
    # Adapted from https://tiao.io/post/tutorial-on-variational-autoencoders-with-a-concise-keras-implementation/
    # I take no responsibility for this class
    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(KLDivergenceLayer, self).__init__(*args, **kwargs)

    def call(self, inputs):

        mu, log_var = inputs

        kl_batch = - .5 * keras.backend.sum(1 + log_var -
                                keras.backend.square(mu) -
                                keras.backend.exp(log_var), axis=-1)

        self.add_loss(keras.backend.mean(kl_batch), inputs=inputs)

        return inputs    

def train_network(epochs,df,hard_evidence,activation_types,hidden_layers,encoding_dim,sizes_sorted,loss_function,training_method,activity_regularizer,input_layer_type,unlabeled_data_percentage,VAE,CNN,kernel_landmarks,CNN_layers,CNN_filters,CNN_kernel_size):
    if training_method=='supervised' or training_method=="unsupervised":
        x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(df,hard_evidence, test_size=0.2) #unsupervised
    elif training_method=="supervised_2_percent":
        x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(df,hard_evidence, test_size=0.98)
    elif training_method=="semi" or training_method=="semi_supervised" or training_method=="semisupervised" or training_method=="semi_sup_first" or training_method=="semi_mixed":
        x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(df,hard_evidence, test_size=unlabeled_data_percentage) # semi supervised
        x_train_nolabel, x_test_nolabel, y_train_nolabel, y_test_nolabel = sklearn.model_selection.train_test_split(x_test,y_test,test_size=0.2) # semi supervised
    else:
        raise Exception("Invalid training method")



    x_train = np.float32(x_train)
    x_test = np.float32(x_test)
    y_train = np.float32(y_train)
    y_test = np.float32(y_test)


    # types = ['relu','relu','relu','relu','relu']
    input_dim = sum(sizes_sorted)
    # this is our input placeholder
#     input_layer = keras.layers.Input(shape=(None,input_dim))
    input_layer = keras.layers.Input(shape=(input_dim,))
    
    


    # "encoded" is the encoded representation of the input
    if input_layer_type =='dense' and not CNN:
        x = keras.layers.Dense(input_dim, activation='relu',activity_regularizer=activity_regularizer)(input_layer)
    elif CNN:
        x = tf.expand_dims(input_layer, axis=2)
        x = keras.layers.Conv1D(input_shape=(0,input_dim),filters=CNN_filters, kernel_size=CNN_kernel_size, activation='relu')(x)
        x = keras.layers.MaxPooling1D(pool_size=2)(x)
        x = keras.layers.Flatten()(x)

    elif input_layer_type == 'gaussian_noise':    
        x = keras.layers.GaussianNoise(0.01)(input_layer)
    elif input_layer_type == 'gaussian_dropout':
        x = keras.layers.GaussianDropout(0.01)(input_layer)
    elif input_layer_type == 'sqrt_softmax':
        x = keras.layers.Lambda(keras.backend.sqrt)(input_layer)
        x= keras.layers.Softmax()(x)
    elif input_layer_type == "gaussian_kernel":
        
        x = gkernel.GaussianKernel3(kernel_landmarks, input_dim)(input_layer)


    encode_ratio=0.1
    middle = hidden_layers//2
    hidden_layer_list = []
    for i in range(hidden_layers):
        if CNN and i < CNN_layers-1:
            
            x = tf.expand_dims(x, axis=2)
            x = keras.layers.Conv1D(filters=CNN_filters, kernel_size=CNN_kernel_size, activation='relu')(x)
            x = keras.layers.MaxPooling1D(pool_size=2)(x)
            x = keras.layers.Flatten()(x)
            
        elif VAE and i==middle:
            
            #Credits to https://tiao.io/post/tutorial-on-variational-autoencoders-with-a-concise-keras-implementation/
            z_mean = keras.layers.Dense(encoding_dim)(x)
            z_log_var = keras.layers.Dense(encoding_dim)(x)
            z_mean, z_log_var = KLDivergenceLayer()([z_mean, z_log_var])
            z_sigma = keras.layers.Lambda(lambda t: keras.backend.exp(.5*t))(z_log_var)

            eps = keras.layers.Input(tensor=keras.backend.random_normal(shape=(keras.backend.shape(x)[0], encoding_dim)))
            z_eps = keras.layers.Multiply()([z_sigma, eps])
            z = keras.layers.Add()([z_mean, z_eps])
            

        else:
            ratio = 2**(math.log2(encode_ratio)+abs(i-middle))
            size = encoding_dim if i==middle else max(( min(input_dim,int(ratio*input_dim))),encoding_dim)
    #         print(size)
            if len(activation_types)>1:
                x = keras.layers.concatenate([keras.layers.Dense(size, activation=type,activity_regularizer=activity_regularizer)(x) for type in activation_types],axis=1)
            else:
                x = keras.layers.Dense(size, activation=activation_types[0],activity_regularizer=activity_regularizer)(x)
        #     x = keras.layers.Dense(min(input_dim,int(ratio*input_dim)), activation='relu')(x)


    final_layer_list= [keras.layers.Dense(size, activation='softmax',activity_regularizer=activity_regularizer)(x) for size in sizes_sorted]

    decoded = keras.layers.concatenate(final_layer_list,axis=1)




    # this model maps an input to its reconstruction
    autoencoder = keras.models.Model(input_layer, outputs=decoded)



    hist = keras.callbacks.History()




    # In[21]:
    if VAE:
#         decoder_h = Dense(intermediate_dim, activation='relu')
#         decoder_mean = Dense(original_dim, activation='sigmoid')
#         h_decoded = decoder_h(z)
#         x_decoded_mean = decoder_mean(h_decoded)
        
#         # end-to-end autoencoder
#         vae = Model(x, x_decoded_mean)

        # encoder, from inputs to latent space
        encoder = keras.models.Model(input_layer, outputs=z_mean)
        


        # generator, from latent space to reconstructed inputs
        decoder_input = keras.layers.Input(shape=(encoding_dim,))
        x = decoder_input
        for i in range(middle+1,hidden_layers):
            ratio = 2**(math.log2(encode_ratio)+abs(i-middle))
            size = encoding_dim if i==middle else max(( min(input_dim,int(ratio*input_dim))),encoding_dim)
    #         print(size)
            if len(activation_types)>1:
                x = keras.layers.concatenate([keras.layers.Dense(size, activation=type,activity_regularizer=activity_regularizer)(x) for type in activation_types],axis=1)
            else:
                x = keras.layers.Dense(size, activation=activation_types[0],activity_regularizer=activity_regularizer)(x)

        final_layer_list_generator = [keras.layers.Dense(size, activation='softmax',activity_regularizer=activity_regularizer)(x) for size in sizes_sorted]
        decoded_generator = keras.layers.concatenate(final_layer_list_generator,axis=1)
        generator = keras.models.Model(decoder_input, outputs=decoded_generator)
        
#         old_loss_func = loss_function
#         loss_function=vae_loss(z_mean, z_log_var,old_loss_func)
#         autoencoder.compile(optimizer='adam',loss=loss_function, metrics=['accuracy'],run_eagerly=True) 
#     else:
    autoencoder.compile(optimizer='adam',loss=loss_function, metrics=['accuracy']) #semi supervised


        

    if training_method=='supervised' or training_method=="supervised_2_percent":
        autoencoder.fit(x_train, y_train, epochs=epochs, batch_size=32, shuffle=True, validation_data=(x_test, y_test), callbacks=[hist],verbose=1)
    elif training_method=="semi" or training_method=="semi_supervised" or training_method=="semisupervised":
        # SEMI SUPERVISED
        autoencoder.fit(x_train_nolabel, x_train_nolabel, epochs=epochs, batch_size=32, shuffle=True, validation_data=(x_test_nolabel, x_test_nolabel), callbacks=[hist],verbose=1)
        autoencoder.fit(x_train, y_train, epochs=epochs, batch_size=32, shuffle=True, validation_data=(x_test, y_test), callbacks=[hist],verbose=1)
    elif training_method=="unsupervised":
        # UNSUPERVISED
        autoencoder.fit(x_train, x_train, epochs=epochs, batch_size=32, shuffle=True, validation_data=(x_test, x_test), callbacks=[hist],verbose=1)
    elif training_method=="semi_sup_first":
        autoencoder.fit(x_train, y_train, epochs=epochs, batch_size=32, shuffle=True, validation_data=(x_test, y_test), callbacks=[hist],verbose=1)
        autoencoder.fit(x_train_nolabel, x_train_nolabel, epochs=epochs, batch_size=32, shuffle=True, validation_data=(x_test_nolabel, x_test_nolabel), callbacks=[hist],verbose=1)
    elif training_method=="semi_mixed":
        for i in range(epochs):
            autoencoder.fit(x_train_nolabel, x_train_nolabel, epochs=1, batch_size=32, shuffle=True, validation_data=(x_test_nolabel, x_test_nolabel), callbacks=[hist],verbose=1)
            autoencoder.fit(x_train, y_train, epochs=1, batch_size=32, shuffle=True, validation_data=(x_test, y_test), callbacks=[hist],verbose=1)
    else:
        raise Exception("Invalid training method")
    return autoencoder
        
def measure_performance(df, hard_evidence, autoencoder,sizes_sorted):
    test_data = df.head(10000)
    verify_data = hard_evidence.iloc[test_data.index]
    results = pd.DataFrame(autoencoder.predict(test_data))



    i=0
    distances_before=[]
    distances_after=[]
    for size in sizes_sorted:
        dist_before = prob_distance(verify_data.iloc[:,i:i+size],test_data.iloc[:,i:i+size])
        dist_after = prob_distance(verify_data.iloc[:,i:i+size],results.iloc[:,i:i+size])
        distances_before.append(np.nansum(dist_before)/len(dist_before))
        distances_after.append(np.nansum(dist_after)/len(dist_after))
        i+=size

    avg_distance_before = np.nansum([sizes_sorted[i]*distances_before[i] for i in range(len(sizes_sorted))])/sum(sizes_sorted)
    avg_distance_after = np.nansum([sizes_sorted[i]*distances_after[i] for i in range(len(sizes_sorted))])/sum(sizes_sorted)
    improvement = avg_distance_before - avg_distance_after
    noise_left = 100*avg_distance_after/avg_distance_before
    return noise_left    

In [ ]:
#-------------
def run_experiment(string="",epochs=epochs_default,use_previous_df=False,BN_size=BN_size_default,sampling_density=sampling_density_default,mu=mu_default,sigma=sigma_default,activation_types=activation_types_default,hidden_layers=hidden_layers_default,encoding_dim=encoding_dim_default,loss_function=loss_function_default,training_method=training_method_default,activity_regularizer=activity_regularizer_default,input_layer_type=input_layer_type_default,unlabeled_data_percentage=unlabeled_data_percentage_default, VAE=VAE_default,CNN=CNN_default,kernel_landmarks=kernel_landmarks_default,CNN_layers=CNN_layers_default,CNN_filters=CNN_filters_default,CNN_kernel_size=CNN_kernel_size_default):
    bn = make_bn(BN_size,sampling_density)
    df, hard_evidence, sizes_sorted = make_df(use_previous_df,bn,mu,sigma)

    if loss_function=='JSD':
        loss_function = lambda y_true,y_pred: custom_loss2(y_true,y_pred,sizes_sorted)

    autoencoder =train_network(epochs,df,hard_evidence,activation_types,hidden_layers,encoding_dim,sizes_sorted,loss_function,training_method,activity_regularizer,input_layer_type,unlabeled_data_percentage, VAE,CNN,kernel_landmarks,CNN_layers,CNN_filters,CNN_kernel_size)
    noise_left = measure_performance(df, hard_evidence, autoencoder,sizes_sorted)
    print(string+ "; " + str(100-noise_left))
    del autoencoder
    gc.collect()
    keras.backend.clear_session()

def all_experiments(experiments):
    for experiment_configs in experiments:
        print("{------- NEW EXPERIMENT ---------}; ")
        for experiment_config in experiment_configs:
#             print(experiment_config)
            run_experiment(**experiment_config)
            
all_experiments(experiments)
